In [707]:
import requests
import json

url = '10.168.20.20:5010'
entries = ['5b491e4dd7f17a0025e48d7a']
rolesa = '5afdb8cf93beb10025c13f4f'

#url = 'localhost:5010'
#entries = ['5b3ec47706f9e50f2f36acc8', '5b3fd54906f9e50f2f36acc9']
#rolesa = '593f49be42d3ed97390f25a7'

In [708]:
pipeline = [
    {'$match': {'_id': {'$in': entries}, 'roles._id': rolesa}},
    {
        '$graphLookup': {
            'from': 'applications',
            'startWith': '$deps._id',
            'connectFromField': 'deps._id',
            'connectToField': '_id',
            'as': 'nodes',
            'maxDepth': 10,
            'depthField': 'steps'
        }
    },
    {'$project': {
                    'name': 1, 'family': 1, 
                    'servers': 1,
                    'deps._id': 1, 'deps.endpoint': 1, 
                    'nodes._id': 1, 'nodes.deps': 1, 'nodes.steps': 1, 'nodes.name': 1, 'nodes.family': 1, 'nodes.servers': 1
                 }
    }
];

jpipeline = json.dumps(pipeline)

data = requests.post('http://%s/aggregate' % url, json={'entity': 'applications', 'pipeline': jpipeline})
data = data.json()

In [709]:
import sys
import os
import numpy as np
cwd = os.getcwd()
spltted = cwd.split('/')[:-1]
sys.path.append('/'.join(spltted))

import networkx as nx
from app.services.gridOrchestrator import GridOrchestrator
from app.libs.network.baseNetwork import BaseNetwork
from app.libs.histograms.histogram import Histogram
from app.libs.histograms.grid import GridHistogram

from app.libs.transformDict import append_servers, transform_dict

from svgwrite import Drawing

network = BaseNetwork()
network.make(data.get('items')).get_graph()

In [710]:
mapping_style = {
    'aws': '.aws{color: #f58536;fill: #c4692c;}',
    'premise': '.premise{color: #5b5b5b;fill: #8e8e8e;}',
    'openstack': '.openstack{color: #ce081f;fill: #af071a;}',
    'azure': '.azure{color: #1566bf;fill: #7396bc;}',
    'default': '.dark{fill: currentColor;}.darker{filter: url(#brightness)}.2darker{filter: url(#darker)}text{font-family:Arial, Helvetica, sans-serif; fill: #672064;}.white{fill:#fff;}'
}


class LoadTemplates(object):
    
    def __init__(self, path):
        self._path = path
        self._mapp = {}
        
        self.crawler()
    
    def __call__(self):
        return self._mapp
        
    def crawler(self):
        for (dirpath, dirnames, filenames) in os.walk(mypath):

            if filenames:
                spl = dirpath.split('/')[-1]
                self.iter_files(spl, filenames)

    
    def without_prefix(self, label):
        spt = label.split('.')[:-1]
        return ''.join(spt)
    
    def get_prefix(self, pfx):
        if pfx:
            return '%s.' % pfx
        
        return ''
    def iter_files(self, prefix, filenames):
        
        for file in filenames:
            
            name = '%s%s/%s' % (self._path, prefix, file)
            key= '%s%s' % (self.get_prefix(prefix), self.without_prefix(file))
            with open(name) as f:
                data = json.load(f)    
                self._mapp[key] = data
        
        

mypath = '/'.join(spltted) + '/assets/symbol/'

mapping = LoadTemplates(mypath)()

In [711]:
class Creator(object):
    def __init__(self, assets):
        self._assets = assets

    def make(self):
        points = self.get_args('points')
              
        if points:
            points = self.getPoints(points)
            self.set_args('points', points)
            
        return self._assets
    
    def get_args(self, label):
        return self._assets.get(label)
    
    def set_args(self, label, value):
        self._assets[label] = value
    
    def getPoints(self, points):
        prepared = []
        splitted = points.split(' ')

        tmp = None
        for item in splitted:
            if tmp:
                cons = (tmp, item)
                prepared.append(cons)
                tmp = None
            else:
                tmp = item
        
        return prepared

class ManageAssets(object):
    def __init__(self, symbol, creator=Creator):
        self._symbol_assets = symbol
        self._creator = creator
    
    def polyline(self, opt={}):
        return self._symbol_assets.dwg.polygon(**opt)
    
    def path(self, opts):
        return self._symbol_assets.dwg.path(**opts)
    
    def create(self, configs):
        objs = []
        items = configs.get('els')
        
        for item in items:
            args = item.get('args')
            shape = item.get('shape')
            opts = self._creator(args).make()
            
            obj = getattr(self, shape)(opts)
            objs.append(obj)
        
        return objs

In [712]:
import math

class CalCosine(object):
    
    cos = math.cos(math.radians(30.4))
    
    @staticmethod
    def cal(cat):
        hip =  cat / CalCosine.cos
        return math.sqrt((hip**2) - (cat**2))


class CalSine(object):
    
    cos = math.sin(math.radians(30.4))
    
    @staticmethod
    def cal(cat):
        hip =  cat / CalSine.cos
        return math.sqrt((hip**2) - (cat**2))
    
class MatrixRotation(object):
    
    def __init__(self, max_x, size, off):
        self._size = size
        self._off = off
        self._mx = (size[0] * off[0]) / 2
        self._cosi = CalCosine.cal(self._mx)
        
        self._rect_y = ((max_x-1) * self._cosi)
        
    def cal_offy(self, x, y):
        return self._rect_y + (y * self._cosi) - (x * self._cosi)
    
    def cal_offx(self, x, y):
        return (y * self._mx) + (x * self._mx)

In [713]:
#10 = 100 / x

#50 = 100 / 2
#50 = 100 / x -> x = 100 / 50
#50 = x / 2 -> x = 50 * 2

In [714]:
from operator import itemgetter

class CalGrid(object):
    def __init__(self, nsize, size, gsize=2):
        
        self._map = None
        self._size = size
        self._gsize = gsize
        
        self._base = self.cal_base(nsize)
        self._cato = self.cal_cosins()
        
    
    def cal_base(self, nsize):
        return [nsize[x]/self._gsize for x in range(2)]
    
    def cal_cosins(self):
        cata = self._size[0] / self._gsize
        cos1 = CalCosine.cal(cata)
        cos2 = CalCosine.cal(self._base[0])
        return cos1, cos2
    
    def get_position(self, pos):
        mapper = self.build_map()
        return [mapper[x][pos] for x in range(2)]  

class CalGridOne(CalGrid):
    def __init__(self, base, size, gsize=2):
        super().__init__(base, size, gsize)
    
    def build_map(self):
        
        if self._map == None:
            xbase, ybase = [self._base[x] for x in range(2)]
            cato_box, cato_node = self._cato
            
            yspace = (self._size[1] / self._gsize)
            
            grid_x = [xbase]
            grid_y = [yspace - (cato_box / 2)]

            self._map = grid_x, grid_y
            
        return self._map
    
class CalGridFourth(CalGrid):
    def __init__(self, base, size, gsize=2):
        super().__init__(base, size, gsize)
     
    def build_map(self):
        
        if self._map == None:
            xbase, ybase = [self._base[x] for x in range(2)]
            cato_box, cato_node = self._cato
            
            yspace = (self._size[1] / self._gsize)
            
            grid_x = (xbase, xbase*2, 0, xbase)
            grid_y = (
                yspace - cato_box, 
                yspace - cato_box + cato_node, 
                yspace - cato_box + cato_node,  
                yspace
            )

            self._map = grid_x, grid_y
            
        return self._map

class CalGridNineth(CalGrid):
    def __init__(self, base, size, gsize=2):
        super().__init__(base, size, gsize)
     
    def build_map(self):
        
        if self._map == None:
            xbase, ybase = [self._base[x] for x in range(2)]
            cato_box, cato_node = self._cato
            
            yspace = (self._size[1] / self._gsize)
            
            grid_x = (xbase, xbase*2, 0, xbase)
            grid_y = (
                yspace - cato_box, 
                yspace - cato_box + cato_node, 
                yspace - cato_box + cato_node,  
                yspace
            )

            self._map = grid_x, grid_y
            
        return self._map

    
class FactoryCalGrid(object):
    
    @staticmethod
    def caller(gsize, base, size):
        grid = (CalGridOne, CalGridOne, CalGridFourth, CalGridNineth)
        
        call = grid[gsize]
        return call(base, size)
        
    
class MicroCalSingle(object):
    
    def __init__(self, space=80):
        self._space = space
    
    def get_pos(self, pos):
        cata = self._space
        cos = CalCosine.cal(cata)
    
        posx = pos[0] + (self._space / 2)
        posy = pos[1] - (cos/2)
        
        return (posx, posy)
        
    def get_size(self, size):
        return (size[0] - self._space, size[1])

    
class PosMatrix3D(object):
    
    def __init__(self, size, pos, off):
        self._size = size
        self._off = off
        self._pos = pos
    
    def __call__(self):
        return self._pos
    
    def __getitem__(self, i):
        return self._pos[i]
        
    def mid_back(self):
        md = self._mid_back()
        
        pos = [self._pos[x] + md[x] for x in range(2)]
        return pos
    
    def start_point_back(self):
        md = self._start_back()
        om = self._mid_space_back()
        
        pos = [self._pos[x] + md[x] + om[x] for x in range(2)]
        return pos
    
    def mid_point_back(self):
        md = self._mid_back()
        om = self._mid_space_back()
        
        pos = [self._pos[x] + md[x] + om[x] for x in range(2)]
        return pos
    
    def end_point_back(self):
        md = self._end_back()
        om = self._mid_space_back()
        
        pos = [self._pos[x] + md[x] + om[x] for x in range(2)]
        return pos
    
    def _start_back(self):
        ss1 = self._size[0] / 2
        cos2 = CalCosine.cal(ss1)
        
        x = 0
        y = self._size[1] - cos2
        
        return (x, y)
    
    def _mid_back(self):
        ss1 = self._size[0] / 4
        cos2 = CalCosine.cal(ss1)
        
        x = self._size[0] / 4
        y = self._size[1] - cos2
        
        return (x, y)
    
    def _mid_space_back(self):
        sizex = (self._size[0]) / 4
        cos = CalCosine.cal(sizex)
        
        return (-sizex, cos)
    
    def _end_back(self):
        ss1 = self._size[0] / 4
        cos2 = CalCosine.cal(ss1)
        
        x = self._size[0] / 2
        y = self._size[1]
        
        return (x, y)
    
    def mid_front(self):
        md = self._mid_front()
        
        pos = [self._pos[x] + md[x] for x in range(2)]
        return pos
    
    def _mid_front(self):
        ss = self._size[0] / 2
        cos1 = CalCosine.cal(ss)
        
        x = self._size[0] - (self._size[0] / 4)
        y = self._size[1] - (cos1 * 1.5)
        
        return (x, y)
    
    def _mid_space_front(self):
        sizex = (self._size[0]) / 4
        cos = CalCosine.cal(sizex)
        
        return (sizex, -cos)
    
        
    def mid_point_front(self):
        md = self._mid_front()
        om = self._mid_space_front()
        
        pos = [self._pos[x] + md[x] + om[x] for x in range(2)]
        return pos
        
    
class Matrix3D(MatrixRotation):

    def rotateXY(self, cad, size=1):
        
        if size > 1:
            nposy = (size - 1) / 2 
            cad = (cad[0], cad[1]+nposy)
        
        res = self.cal_off(cad)
        
        return PosMatrix3D(self._size, res, self._off)

    def rotateNodeXY(self, node):
        cadx, cady, size, _ = node
        return self.rotateXY((cadx, cady), size)
        
    def cal_off(self, pos):
        x = self.cal_offx(*pos)
        y = self.cal_offy(*pos)
        
        return (x, y)
    

class HelperDraw(object):
    
    def __init__(self, size, matrix3d=None):
        self._size = size
        self._matrix3d = matrix3d
        
class HelperDrawBasePolyline(HelperDraw):
        
    def create_polyline_by_pos(self, cad1, cad2):
        pos1 = self._matrix3d.rotateXY(cad1)()
        pos2 = self._matrix3d.rotateXY(cad2)()
        
        return self.append_points(pos1, pos2)
        
    def cal_points(self, pos, pos2):
        sizex = self._size[0]
        h_sizex = sizex / 2
        sizey = self._size[1]
        
        cos = CalCosine.cal(h_sizex)
        
        lst = [
            (pos[0], pos[1] + sizey - cos),
            (pos[0] + h_sizex, pos[1] + sizey - (cos*2)),
            (pos2[0] + sizex, pos2[1] + sizey - cos),
            (pos2[0] + h_sizex, pos2[1] + sizey)
        ]
        
        return lst
    
    def append_points(self, pos1, pos2):
        return self.cal_points(pos1, pos2)

class HelperDrawLabel(HelperDraw):
    
    def label_by_node(self, pos, node, maxl=10):
        name = node.get('name', '-')
        
        name = name[0:maxl]
        ln = len(name)
        
        fsize = self._size[0] * 0.18
        sposy = self._size[1] * .6
        
        sposx = (self._size[1] * 0.57) - (ln * (fsize * 0.16))
        
        npos = [-1*pos[x] for x in range(2)]
        npos = (npos[0]+sposx, npos[1]+sposy)
        
        degrees = (30.404, -40.)
        transf = (*pos, *degrees, *npos)
        
        opts = {
            'font_size': fsize,
            'transform': 'translate(%s, %s) skewY(%s) skewX(%s) translate(%s, %s)' % transf}
        
        return name, pos, opts

class ScoreSize(object):

    mindex = [2, 2.5, 3.5, 7, 11, 22, 46, 76]
    mvalue = ['nano', 'micro', 'small', 'medium', 'large', '2xlarge', '4xlarge', '8xlarge']
    
    @staticmethod
    def scored(scr):
        valued = None
        
        for key, val in zip(ScoreSize.mindex, ScoreSize.mvalue):
            if scr <= key:
                valued = val
                break             
 
        return valued

class ScoreServer(object):
    
    @staticmethod
    def make_score(cpu, memory):
        return (float(cpu) * 1.5) + float(memory)
    
    @staticmethod
    def val_score(score):
        return ScoreSize.scored(score)
    
    
class DcServers(object):
    
    allowed = ['aws', 'openstack', 'azure']
    
    @staticmethod
    def byServer(server, dft='premise'):
        
        if 'datacenters' in server:
            dc = server.get('datacenters')
            
            if 'provider' in dc:
                provider = dc.get('provider').lower()

                if provider in DcServers.allowed:
                    return provider
                
        return dft
    
class HelperDrawApplication(HelperDraw):
    
    def __init__(self, size, servers, microCal=MicroCalSingle):
        
        ajust = size[0]*0.16
        self._microsingle = microCal(ajust)
        
        self._apps = []
        self._servers = servers
        self._calGrid = None
        self._pos = None
        self._nsize = None
        self._gsize = None
        
        super().__init__(size)
    
    def node_size(self, prop):
        if prop == 1:
            prop += 1
        self._nsize = (self._size[0] / prop, self._size[1] / prop)
        
    def grid_size(self, qtd):
        self._gsize = math.ceil(math.sqrt(qtd))
        self.node_size(self._gsize)
    
    def execute(self, pos, node):
        self._pos = pos
        servers = node.get('servers')
        
        family = node.get('family', 'application').lower()
        
        if servers:
            self.template_with_servers(servers, family)
            
        if not servers:
            self.template_without_servers(family, node)
        
    def template_with_servers(self, servers, family):
        qtd = len(servers)
        
        self.grid_size(qtd)

        self._calGrid = FactoryCalGrid.caller(self._gsize, self._nsize, self._size)

        ordered_servers = self.sorted_servers(servers, family)
        for serv in enumerate(ordered_servers):
            self.single_server(*serv)
            
    def sorted_servers(self, servers, family):
        
        prepared = []
        
        for id_server in servers:
            details = self._servers.get(id_server)
            
            cpu = details.get('cpu', 1)
            memory = details.get('memory', 1)
            score = ScoreServer.make_score(cpu, memory)
            
            
            ss = {k: details.get(k, None) for k in ('hostname', 'ipv4_private', 'ipv4_public', 'datacenters', 'services', 'storage', 'cpu', 'memory', 'environment', 'role', 'os')}
            ss['asset'] = '%s.%s' % (family, ScoreServer.val_score(score))
            ss['score'] = score
            
            prepared.append(ss)
        
        prepared = sorted(prepared, key=itemgetter('score'), reverse=True)
        return prepared
        
    def template_without_servers(self, family, node):
        step = 1
        counter = 0
        
        catsize = node.get('size', 'medium')
        asset = '%s.%s' % (family, catsize)
        template = 'aws'
        
        self.grid_size(step)
        self._calGrid = FactoryCalGrid.caller(self._gsize, self._nsize, self._size)
        self.draw_applications(counter, asset, template)
        
    def single_server(self, counter, server):
        asset = server.get('asset')
        score = server.get('score')

        template = DcServers.byServer(server)
        self.draw_applications(counter, asset, template)
        
    def draw_applications(self, counter, asset, template):
        xpos, ypos = self._calGrid.get_position(counter)
        
        npos = (self._pos[0] + xpos, self._pos[1] + ypos)
        self.draw_symbol(npos, asset, template)
        
    def draw_symbol(self, pos, asset, template):
        npos = self._microsingle.get_pos(pos)
        nsize = self._microsingle.get_size(self._nsize)
        
        app = (asset, template, npos, nsize)
        self._apps.append(app)
    
    def get_apps(self):
        return self._apps

    
class PathFactor(object):
    
    def __init__(self):
        self._d = "M"
        
    def __call__(self):
        return self._d.strip()
    
    def add_point(self, x, y):
        self._d += "%s %s " % (x, y)
        
    
class HelperDrawConnector(HelperDraw):
    def __init__(self, size, matrix3d=None, pathFactor=PathFactor):
        super().__init__(size, matrix3d)
        
        self._path = pathFactor()
        
    def connect(self, node1, node2):
        
        x1, y1, s1, _ = node1
        x2, y2, s2, _ = node2
        
        dx = (x2-x1) - 1
        dy = (y2-y1)
        print(dx, dy)
        
        self.start_c((x1, y1), s1, dy)
        
        if dx >= 1: 
            self.point_f((x1, y1), 1)
            self.node_f((x1+dx, y1), 1)
            
            
            #self.point_e_b((x1+dx, y1), 1)
        
        self.end_c((x2, y2), s2)
        
        
        d = self._path()
        
        return d
    
    def start_c(self, pos, size, dy):
        PosMatrix = self._matrix3d.rotateXY(pos, size)
        self.node_mid_front(PosMatrix)
        
        if dy > 0:
            self.point_mid_front(PosMatrix)
        
        if dy == 0:
            self.point_mid_front(PosMatrix)
            
        if dy < 0:
            self.point_mid_front(PosMatrix)
        
        
        self.point_mid_front(PosMatrix)
    
    def node_f(self, pos, size):
        PosMatrix = self._matrix3d.rotateXY(pos, size)
        self.node_mid_front(PosMatrix)
        
    def point_f(self, pos, size):
        PosMatrix = self._matrix3d.rotateXY(pos, size)
        self.point_mid_front(PosMatrix)
    
    def point_b(self, pos, size):
        PosMatrix = self._matrix3d.rotateXY(pos, size)
        self.point_mid_front(PosMatrix)
        
    def point_e_b(self, pos, size):
        PosMatrix = self._matrix3d.rotateXY(pos, size)
        self.point_end_back(PosMatrix)
        
    def end_c(self, pos, size):
        PosMatrix = self._matrix3d.rotateXY(pos, size)
        self.node_mid_back(PosMatrix)
        self.point_mid_back(PosMatrix)
    
    
    # front end points
    def node_mid_front(self, PosMatrix):
        pos = PosMatrix.mid_front()
        self._path.add_point(*pos)
    
    def point_mid_front(self, PosMatrix):
        pos = PosMatrix.mid_point_front()
        self._path.add_point(*pos)
    
    
    
    # back end points
    def node_mid_back(self, PosMatrix):
        pos = PosMatrix.mid_point_back()
        self._path.add_point(*pos)
    
    def point_start_back(self, PosMatrix):
        pos = PosMatrix.start_point_back()        
        self._path.add_point(*pos)
        
    def point_mid_back(self, PosMatrix):
        pos = PosMatrix.mid_back()        
        self._path.add_point(*pos)
        
    def point_end_back(self, PosMatrix):
        pos = PosMatrix.end_point_back()        
        self._path.add_point(*pos)
        
class DrawTemplateSVG(object):
    def __init__(self, tmax, servers, darea=DrawArea, symbols=SymbolAssets, m3d=Matrix3D):
        self._off = (2, 2.5)
        size = 100
        self._size = (size, size*1.9)

        self._matrix3d = m3d(tmax[0], self._size, self._off)
        
        self._area = darea(self._off, self._size, tmax[0], tmax[1]).area()

        self.dwg = Drawing('test.svg', size=self._area)
        self._symbols = symbols(self.dwg)
        self._servers = servers
        
        self.setup()
    
    def setup(self):
        symbol = self._symbols.square((0,0), size=self._area, opts={'fill': "#ededed"})
        self.add(symbol)
        
        self._symbols.brightness()
        self._symbols.brightness("darker", 0.4)
        
        symbol = self._symbols.polyline([(0, 30.75), (157.21, 0), (104.81, 92.25), (63.79, 54.82)], opts={'fill': "#000"})
        viewbox = (0, 0, 157.21, 92.25)
        
        s = self._size[0] / 8
        size = (s, s)
        insert = (152, 3)
        
        opts = {
            'insert': insert, 
            'size': size, 
            'markerUnits': 'strokeWidth',
        }
        
        
        self._symbols.create_marker('arrow', viewbox, [symbol], opts)
    
    def boundary_box(self, pos):
        symbol = self._symbols.asset('boundaries_box.front', 'default', (pos[0], pos[1]), self._size)
        self.add(symbol)
        
    def draw_label(self, pos, node):
        
        text = HelperDrawLabel(self._size, self._matrix3d)\
                        .label_by_node(pos, node)
        
        symbol = self._symbols.text(*text)
        self.add(symbol)
        
    def draw_app(self, item):
        cad1 = [item[x] for x in range(2)]
        node = item[3]
        
        pos = self._matrix3d.rotateNodeXY(item)()
    
        self.draw_grid_size(cad1, item[2])
        self.grid_box(pos)
        self.draw_execute(pos, node)
        self.boundary_box(pos)
        self.draw_label(pos, node)
    
    def draw_execute(self, pos, node):
        hDrawApp = HelperDrawApplication(self._size, self._servers)
        hDrawApp.execute(pos, node)
        
        pSymb = hDrawApp.get_apps()
        
        for symb in pSymb:
            symbol = self._symbols.asset(*symb)
            self.add(symbol)
    
    def grid_box(self, pos, opts={'fill-opacity': '0.4'}):
        symbol = self._symbols.asset('grid.base', 'default', pos, self._size, opts)
        self.add(symbol)
        
    
    def draw_grid_size(self, cad1, size):

        cad2 = (cad1[0], cad1[1]+size-1)
        
        points = HelperDrawBasePolyline(self._size, self._matrix3d)\
                    .create_polyline_by_pos(cad1, cad2)
        
        symbol = self._symbols.polyline(points, {'fill': '#ccc', 'fill-opacity': 0.2})
        self.add(symbol)
    
        
    def add(self, symbol):
        self.dwg.add(symbol)
        
    def save(self):
        self.dwg.save()
        return self.dwg.get_xml()
        
    def draw_connect(self, node1, node2, details={}):
        opts = {
            'stroke_width': "1",
            'stroke': "black",
            'fill': "none",
            "marker_end": self._symbols.get_symbol('arrow').get_funciri()
        }
        
        d = HelperDrawConnector(self._size, self._matrix3d).connect(node1, node2)
        
        symbol = self._symbols.path(d, opts)
        self.add(symbol)

    
    
class DrawLayout(object):
    def __init__(self, grid, index, servers={}, draw=DrawTemplateSVG):
        
        self._grid = grid
        self._index = index
        
        self._max_x = max(self._grid, key=int)
        self._max_y = max(self._grid[0], key=int)
        self.setup_drawer(draw, servers)
        

    def setup_drawer(self, draw, servers):
        tmax = (self._max_x, self._max_y)
        self.drawer = draw(tmax, servers)
        
    def draw_nodes(self):
        
        for col_k in range(self._max_x, -1, -1):
            if col_k in self._grid:
                self.draw_nodes_columm(col_k)
                
        return self
    
    def draw_nodes_columm(self, col_k):
        columm = self._grid[col_k]
        max_y = max(self._grid[col_k], key=int)

        for line_k in range(0, max_y+1):
            if line_k in self._grid[col_k]:
                self.draw_nodes_line(col_k, line_k)
        
    def draw_nodes_line(self, col_k, line_k):
        label = self._grid[col_k][line_k]
                    
        if label in self._index:
            item = self._index[label]
            self.drawer.draw_app(item)
        else:
            pass
            #self.drawer.draw_grid((col_k, line_k))
        
        
        
    def draw_connections(self, edges):
        for edge in edges:
            edg = [self._index[edge[x]] for x in range(2)]
            
            self.drawer.draw_connect(*edg, edge[2])
            
        return self
    
    def save(self):
        return self.drawer.save()

In [715]:
from IPython.display import SVG, display
from functools import reduce
from app.repository.externalMaestroData import ExternalMaestroData

print("-------------------------")
Orchestration = GridOrchestrator(network.graph)
Orchestration.create(entries)

servers_id = reduce(append_servers, Orchestration.get_grid().get_index().values(), [])
servers_id = list(set(servers_id)) #remove duplicate
query = {"_id": servers_id}
ExternalRequest = ExternalMaestroData(owner_id=rolesa)
result = ExternalRequest.get_request(path="servers", query=query)

servers = transform_dict(result)

Layout = DrawLayout(Orchestration.get_grid().get_grid(), Orchestration.get_grid().get_index(), servers)

Layout.draw_nodes()
Layout.draw_connections(network.graph.edges(data='endpoint'))
Layout.save()
   
display(SVG(filename='test.svg'))

-------------------------
0 1
2 -2
1 -1
1 -3
2 1
0 0
0 1
1 -1
0 0
0 0
0 1
0 3
